# Data science tips
## 판다스 코드 속도 최적화를 위한 안내서

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div> 

<div style="text-align: right"> Initial upload: 2020.11.21</div> 
<div style="text-align: right"> Last update: 2020.11.21</div> 

- 참고자료  
    - https://aldente0630.github.io/data-science/2018/08/05/a-beginners-guide-to-optimizing-pandas-code-for-speed.html
    - https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6  
    - https://github.com/s-heisler/pycon2017-optimizing-pandas

판다스 속도 최적화를 위한 방법에 대하여 소개한다. 참고자료의 내용을 실행한 것이다.

pandas 데이터프레임에 함수를 적용하는 방법 중에 느린 것부터 나열하면 다음과 같다.  
- indices를 활용한 row looping  
- iterrrows()를 활용한 looping  
- apply()를 활용한 looping  
- Pandas series를 활용한 vectorization  
- NumPy arrays를 활용한 vectorization  

### 1. 라이브러리 import, 데이터 읽기

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import datetime
import warnings; warnings.filterwarnings('ignore')
plt.style.use('ggplot')
%matplotlib inline

from math import *

In [2]:
url = 'https://raw.githubusercontent.com/sguys99/datasets/master/new_york_hotels.csv'
df = pd.read_csv(url, encoding = 'cp1252')

In [3]:
df.head()

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900


In [4]:
df.shape

(1631, 11)

### 2. Haversine 함수  
두 점위 위도와 경도를 취하여 지구의 곡률을 구하고 그사이 직선거리를 계산하는 함수이다.

In [5]:
def haversine(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

### 3. 비교해보기

#### 단순한 루핑

판다스의 구조는 데이터프레임과 시리즈 두가지로 제공된다. 여기서 데이터프레임은 2차원 배열이다. 다시 말에 열에는 열 이름이, 행에는 인덱스용 레이블이 붙어있는 행렬이다. 시리즈는 축 레이블이 있는 1차원 배열이다.  

In [6]:
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        distance_list.append(d)
    return distance_list

In [7]:
%%timeit
# Haversine 반복 함수 실행하기
df['distance'] = haversine_looping(df)

390 ms ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


- 1600개 행을 연산하는데 390ms가 소요되었다.

#### iterrrows를 사용한 방법

반복문을 반드시 써야한다면, iterrows() 메서드를 사용한다.  
iterrows() 메서드는 데이터프레임의 행을 반복하여 각행의 index와 row 값들을 반환한다.  
iterrows()는 데이터프레임에 최적화되어 있어 수동으로 루프를 도는 것보다 속도가 빠르다.

In [8]:
%%timeit
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985, 
                                      row['latitude'], row['longitude']))
    
df['distance'] = haversine_series

162 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### apply를 사용한 방법

apply() 메서드를 사용하는 방법이 iterrows() 메서드를 사용하는 방법보다 더 좋은 옵션이다. 내부 최적화를 다양하게 활용하기 때문이다.  
*익명의 람다 함수를 사용하여 Haversine 함수를 각 행에 적용하며 각 행의 특정 셀을 함수 입력값으로 지정할 수 있다. 람다 함수는 판다스가 행(축 = 1)과 열(축 = 0) 중 어디에 함수를 적용할지 정할 수 있게 축 매개 변수를 마지막에 포함한다*

In [9]:
%%timeit

df['distance'] = df.apply(lambda row: haversine(40.671, -73.985,
                           row['latitude'], row['longitude']),axis = 1)

59 ms ± 8.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### pandas series를 사용한 벡터화  
데이터프레임과 시리즈는 모두 배열기반이다. 개별 값(스칼라)마다 순차적으로 작동하는 방법대신 벡터화를 사용하면 속도개선 효과를 볼수 있다.  
벡터화(vectorization) 배열 단위로 작업을 실행하는 프로세스이다.

In [10]:
%%timeit

df['distance'] = haversine(40.671, -73.985, 
                            df['latitude'], df['longitude'])

1.64 ms ± 184 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


- apply()와 비교해서 벡터화가 속도가 더 우수한 이유는 함수를 단 한번만 사용했기 때문이다.

#### numpy를 사용한 벡터화

속도가 우선이라면 numpy의 ndarrays 상에서 연산이 작동하도록 구현할 수 있다.  
numpy라이브러리는 사전 컴파일된 c코드로 작업을 수행하기 때문에, indexing, 데이터 type 확인과 같은 pandas series 작업으로 인한 오버헤드가 덜 발생한다.

In [11]:
%%timeit

df['distance'] = haversine(40.671, -73.985, 
                           df['latitude'].values, df['longitude'].values)

158 µs ± 9.05 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
